1. Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import sklearn
from scipy.stats import loguniform, randint, uniform
from sklearn.linear_model import Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import make_scorer

2. Define functions to load data, create features, create target, and scoring function.

In [2]:
def create_features(df):
    df['Spread'] = df['High'] - df['Low']
    df['Gap'] = df['Open'] - df['Close'].shift(1)
    df['Intraday'] = df['Open'] - df['Close']
    return df

def drop_features(df):
    df.drop(columns=['Spread',
                     'Gap',
                     'Intraday',
                     ],
            inplace=True)

    df.drop(columns=['Open','High','Low','Close','Volume','Adj Close',
                     ], inplace=True)
    return df

def process_features(df, lookback, step):
    for i in range(step, lookback+1, step):
        df['%d Spread' % (i)] = df['Spread'].pct_change(periods=i, fill_method=None)
        df['%d Rolling Avg Spread' % (i)] = df['Spread'].rolling(window=i).mean()

        df['%d Gap' % (i)] = df['Gap'].pct_change(periods=i, fill_method=None)
        df['%d Rolling Avg Gap' % (i)] = df['Gap'].rolling(window=i).mean()

        df['%d Intraday' % (i)] = df['Intraday'].pct_change(periods=i, fill_method=None)
        df['%d Rolling Avg Intraday' % (i)] = df['Intraday'].rolling(window=i).mean()
    return df

def features(df, lookback, step):
    create_features(df)
    process_features(df, lookback, step)
    drop_features(df)
    return df

def create_target(df, lookforward=2, target='Open'):
    df['Target'] = np.log(df[target].shift(periods=-lookforward)/df[target].shift(periods=-1))
    return df

def custom_score(y_true, y_pred):
  pred_sign = np.sign(y_pred)
  y_true = np.squeeze(y_true)
  returns = np.where((pred_sign == 1), y_true, 0)
  return returns.mean()

custom_scorer = make_scorer(custom_score, greater_is_better=True)

3. Define the models we are going to use

In [3]:
estimator1 = Ridge()
estimator2 = Lasso(alpha=.001)
estimator3 = KNeighborsRegressor()
models = [estimator1,estimator2,estimator3]
estimator = VotingRegressor(estimators=[('Ridge', estimator1),
                                        ('Lasso', estimator2),
                                        ('KNN', estimator3),
                                        ])

4. Define target, cross validation folds, interval, and lookback parameters.

In [4]:
lookforward = 2
tscv = TimeSeriesSplit(n_splits=5, gap=lookforward)
step = 2
lookback = 2

5. Load data

In [5]:
spy = yf.download('SPY', start='2004-01-01')
agg = yf.download('AGG', start='2004-01-01')

spy = create_target(spy, lookforward, target='Open')

spy = features(spy, lookback, step)
spy = spy.add_suffix(' SPY')
agg = features(agg, lookback, step)
agg = agg.add_suffix(' AGG')
cv = pd.merge(spy, agg, how='inner', on='Date')

cv.drop(cv.tail(lookforward).index, inplace=True)
cv.drop(cv.head(lookback).index, inplace=True)
X = cv
y = X[['Target SPY']]
X = X.drop(columns=['Target SPY'])
X.fillna(method="ffill", inplace=True)
X.replace([np.inf, -np.inf], 0, inplace=True)
X.fillna(0, inplace=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


6. Define parameter grid. (Look at models on sklearns docs to find which parameters of a model you can change)

In [6]:
param_grid = {
    'Ridge__alpha': loguniform(1e-5, 1e0),
    'Lasso__alpha': loguniform(1e-5, 1e0),
    'KNN__n_neighbors': randint(1,10),
}

7. Split train and test data and run Random Search on train data

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.85, shuffle=False)
search = RandomizedSearchCV(estimator, param_distributions=param_grid, n_iter=100, cv=tscv, scoring=custom_scorer, n_jobs=-1, verbose=1)
search.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=TimeSeriesSplit(gap=2, max_train_size=None, n_splits=5, test_size=None),
                   estimator=VotingRegressor(estimators=[('Ridge',
                                                          Ridge(alpha=0.31856512010970556)),
                                                         ('Lasso',
                                                          Lasso(alpha=0.0011755154435882887)),
                                                         ('KNN',
                                                          KNeighborsRegressor(n_neighbors=4))]),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'KNN__n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017957BC6D00>,
                                        'Lasso__alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017957BE3760>,
                                        'Ridge__alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000017957BC8430>},
                   scoring=make_scorer(custom_score), verbose=1)

8. Print best parameters

In [12]:
print(search.best_params_)
print(search.best_score_)

{'KNN__n_neighbors': 8, 'Lasso__alpha': 0.002999525583333498, 'Ridge__alpha': 0.002840026017965097}
0.00035391068997479727


9. Backtest parameters

In [9]:
# !pip3 install backtesting
from backtesting import Strategy, Backtest
from sklearn.model_selection import train_test_split

Loading BokehJS ...

In [13]:
step = 2
lookback = 2

estimator1 = Ridge(alpha=0.002840026017965097)
estimator2 = Lasso(alpha=0.002999525583333498)
estimator3 = KNeighborsRegressor(n_neighbors=8)
models = [estimator1,
          estimator2,
          estimator3,
          ]
estimator = VotingRegressor(estimators=[('Ridge', estimator1),
                                        ('Lasso', estimator2),
                                        ('KNN', estimator3),
                                        ],)

X_test = X_test.iloc[(abs(lookforward)):]
y_test = y_test.iloc[(abs(lookforward)):]

estimator.fit(X_train, y_train)
forecasted = estimator.predict(X_test)

data = yf.download('SPY', start='2004-01-01')
data.drop(data.tail(lookforward).index,inplace=True)
data.drop(data.head(lookback).index,inplace=True)
data = data.iloc[(-X_test.shape[0]):]
data['forecastedValue'] = forecasted
prediction = data

class MyStrategy(Strategy):
    Data = prediction

    def init(self):
        super().init()

    def next(self):
        if self.data.forecastedValue < 0:
            self.sell()
        elif self.data.forecastedValue > 0:
            self.buy()


bt = Backtest(prediction, MyStrategy,
              cash=1000,
              trade_on_close=False,
              exclusive_orders=True
              )
print(bt.run())

[*********************100%***********************]  1 of 1 completed
Start                     2020-01-24 00:00:00
End                       2022-11-16 00:00:00
Duration                   1027 days 00:00:00
Exposure Time [%]                   99.718706
Equity Final [$]                  1311.039856
Equity Peak [$]                   1416.379425
Return [%]                          31.103986
Buy & Hold Return [%]               20.281664
Return (Ann.) [%]                   10.074481
Volatility (Ann.) [%]               24.580116
Sharpe Ratio                         0.409863
Sortino Ratio                        0.666017
Calmar Ratio                         0.572078
Max. Drawdown [%]                  -17.610312
Avg. Drawdown [%]                   -3.194534
Max. Drawdown Duration      250 days 00:00:00
Avg. Drawdown Duration       34 days 00:00:00
# Trades                                  709
Win Rate [%]                        53.314528
Best Trade [%]                       6.348897
Worst Trade

Surprisingly, the results are better than the previous notebook, despite that one cheating a little bit. The strategy beats buying and holding which is good. But would I run this strategy? No, but this is a good starting point.

Somethings to explore futher: Create more features, add more data sources, evaluate more models, evaluate more parameters, evaluate higher period step interval and look back periods, evaluate how many cross validation folds are optimal when taking the bias-variance trade-off into account, backtest with commission, etc.